### Solve a Boggle Game

Given an Boggle arrangement (nxn) letters, find all valid m letter words in a dictionary.

See [The example problem](http://www.geeksforgeeks.org/boggle-find-possible-words-board-characters/)

In [1]:
debugging = False
#debugging = True

logging = True

def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))

def log(f, *args):
    if logging:
        print((f).format(*args))
        
def logError(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))
        
def className(instance):
    return type(instance).__name__

In [2]:
class TrieDict(object):
    
    def __init__(node):
        node.children = {}
        node.endWord = False
        
    def insert(node, word):
        for ch in word:
            if ch in node.children:
                node = node.children[ch]
            else:
                newn = TrieDict()
                node.children[ch] = newn
                node = newn
        node.endWord = True
        
    def nextNode(node, ch):
        return None if ch not in node.children else node.children[ch]
        

In [3]:
class BoggleCube(object):
    """ A n x n matrix of letters """
    def __init__(self, letters):
        self.letters = letters
        self.n = len(letters)
        
    def ch(self, pos):
        n = self.n
        row = pos[0]
        col = pos[1]
        if row < 0 or col < 0 or row >= n or col >= n: 
            return None
        return self.letters[row][col]
        
    def moves(self, pos):
        """ returns array of (ch, pos) tuples """
        r = []
        rangeVals = [-1, 0, 1]
        possibilities = [(pos[0]+row, pos[1]+col) for row in rangeVals for col in rangeVals]
        for p in possibilities:
            if p != pos: # remove 0,0 center.
                ch = self.ch(p)
                if ch:
                    r.append( (ch, p) )
        return r

    def rootMoves(self):
        """ returns array of (ch, pos) tuples """
        r = []
        for pos in [(row, col) for row in range(self.n) for col in range(self.n)]:
            r.append( (self.ch(pos), pos) )
        return r


In [4]:
class Solver(object):
    
    def __init__(self, dictRoot, cube, minWord=3):
        self.results = {}
        self.cube = cube
        self.root = dictRoot
        self.minWord = minWord

    def findMoves(self, currentWord, pts, move, trieNode):
        ch = move[0]
        pos = move[1]
        nextNode = trieNode.nextNode(ch)
        if nextNode:
            currentWord += ch
            if nextNode.endWord:
                dbg('{0} FOUND {1} at {2}', '--'*len(currentWord), currentWord, move)                 
                if len(currentWord) >= self.minWord:
                    # Increment the count of times the word was discovered.
                    if currentWord not in self.results:
                        self.results[currentWord] = 0
                    self.results[currentWord] += 1
            for mv in self.cube.moves(pos):
                dbg('{0} "{1} {2}', '--'*len(currentWord), currentWord, mv) 
                if pos not in pts: # Don't allow letter reuse.
                    self.findMoves(currentWord, pts + [pos], mv, nextNode)
        
    def solve(self):
        for move in self.cube.rootMoves():
            dbg('- ROOT: {0}', move)
            self.findMoves('', [], move, self.root)
        
        return self.results

In [5]:
dictRoot = TrieDict()
for word in ['tit', 'bell', 'cat', 'cattle', 'at', 'a', 'bet', 'bat',
             'battle', 'cab', 'jot', 'jab', 'belt', 'let', 'tell', 
             'boat', 'tab', 'oat', 'toad', 'tot', 'trot', 'eat',
             'east', 'eel', 'seal', 'lead', 'lee', 'tat', 'lab',
             'lot', 'motor', 'debt', 'jolt', 'bomb', 'rot']:
    dictRoot.insert(word)
    

In [6]:
cube = BoggleCube([['c', 'e', 'l', 'd'],
                   ['b', 'a', 't', 'e'],
                   ['j', 'o', 't', 'b'],
                   ['r', 'l', 'm', 'o']])
svr = Solver(dictRoot, cube)
svr.solve()

{'bat': 2,
 'battle': 2,
 'belt': 2,
 'bet': 3,
 'boat': 2,
 'bomb': 2,
 'cab': 1,
 'cat': 2,
 'cattle': 2,
 'debt': 2,
 'eat': 2,
 'jab': 1,
 'jolt': 1,
 'jot': 2,
 'lab': 1,
 'let': 3,
 'lot': 2,
 'motor': 1,
 'oat': 2,
 'rot': 2,
 'tab': 2,
 'tat': 4,
 'tot': 5}